In [1]:
import pandas as pd
import numpy as np
import os
import random
from datetime import datetime, timedelta
import yaml

In [2]:
import sys
def add_row(df,element):
    df = pd.concat([df,pd.DataFrame.from_records([element])])
    return df

lines=[]
skip_lines=0
lines=[]
with open('conf_project.yaml') as infile:
    lines=infile.readlines()
print(lines)


['#centralized model\n', 'dpi: 800\n', 'node_size: 100\n', 'find_min_drones: True\n', 'minmax: "minimize"\n', '#Nokia Flexi Multiradio 10 System Module - Interface OBSAI RP3-01 provide up to 6.144 Gbps of bandwidth for high-speed data transfer.\n', '#https://www.mti-mobile.com/wp-content/uploads/2012/10/6Gbit_OBSAI_Demonstration.pdf\n', '#https://cosconor.fr/GSM/Divers/Equipment/Nokia/Airscale%20system%20module%20product%20description.pdf (pagina 36 - rf1..6)\n', 'Max_Capacity_BS: 34000 #Mbps (6 Gbps)\n', 'Workloads_rs: [[[500.0, 475.0], [500.0, 435.0], [500.0, 415.0], [500.0, 425.0], [500.0, 465.0], [500.0, 405.0], [500.0, 495.0], [500.0, 410.0], [500.0, 440.0], [500.0, 490.0], [500.0, 470.0], [500.0, 475.0], [500.0, 475.0], [500.0, 475.0], [500.0, 470.0], [500.0, 490.0], [500.0, 445.0], [500.0, 405.0], [500.0, 425.0], [500.0, 415.0], [500.0, 475.0], [500.0, 485.0], [500.0, 435.0], [500.0, 405.0], [500.0, 430.0], [500.0, 490.0], [500.0, 435.0], [500.0, 405.0], [500.0, 450.0], [500.0, 

In [3]:
starting_date = "04/05/2023"
def generate_df(users,starting_hours,hours_per_day,work):
        df = pd.DataFrame(columns = ['users', 'work_1', 'work_2', 'work_3', 'date','total'])
        step=0
        entered=False
        hours_per_day=hours_per_day%24
        for i in range(hours_per_day+1):
            work_now = users[i][0]*users[i][1]
            tot_req=work[i][0]+work[i][1]+work[i][2]
            print(work_now,(users[i][0],users[i][1]),tot_req,work_now/tot_req)
            try:
                if(entered):
                    date=datetime.strptime(starting_date+" "+str(starting_hours+step)," %d/%m/%Y %H") +timedelta(1)
                    append={"users": users[i][0],'work_1':work[i][0], 'work_2':work[i][1], 'work_3':work[i][2], 'date':date,'total':work_now/tot_req}
                    df = add_row(df,append)
                    step+=1
                else:
                    date=datetime.strptime(starting_date+" "+str(starting_hours+i), "%d/%m/%Y %H")
                    append={"users": users[i][0],'work_1':work[i][0], 'work_2':work[i][1], 'work_3':work[i][2], 'date':date,'total':work_now/tot_req}
                    df = add_row(df,append)
            except:
                starting_hours = 0
                if(not entered):
                    print(users[i][0],work[i][0],work[i][1],work[i][2],date,work_now,tot_req)
                    date=datetime.strptime(starting_date+" "+str(starting_hours+step), "%d/%m/%Y %H") +timedelta(1)
                    append={"users": users[i][0],'work_1':work[i][0], 'work_2':work[i][1], 'work_3':work[i][2], 'date':date}
                    df = add_row(df,append)
                    step=1
                    entered=True
        v1 = 20
        v2 = 40
        v3 = 55
        def create_R(work,avg,type):
            reqs = []
            tot=avg
            for i in range(work):
                w1 = tot-((random.randrange(1, v1) * tot) / 100)
                if(type==0):
                    reqs.append([avg,w1])  
                if(type==1):
                    w2 = w1-((random.randrange(v1+1, v2) * w1) / 100)
                    reqs.append([avg,w1,w2])  
                if(type==2):
                    w2 = w1-((random.randrange(v1+1, v2) * w1) / 100)
                    w3 = w2-((random.randrange(v2+1, v3) * w2) / 100)
                    reqs.append([avg,w1,w2,w3])
            return reqs,
    
        Workloads_rs=[]
        for i in range(len(df)):
            avg = df.iloc[i]["total"]
            reqs = create_R(df.iloc[i]["work_1"],avg,0) + create_R(df.iloc[i]["work_2"],avg,1) + create_R(df.iloc[i]["work_3"],avg,2)
            Workloads_rs.append(reqs)
        tot=sum([sum(x[:-1]) for x in Workloads_rs[0][0]]) + sum([sum(x[:-1]) for x in Workloads_rs[0][1]]) + sum([sum(x[:-1]) for x in Workloads_rs[0][2]])
        
        return df, Workloads_rs[0][0]+Workloads_rs[0][1]+Workloads_rs[0][2],tot

In [4]:
#set hours_per_day always to 0.
res, work, tot=generate_df(users=[(10000,5)],starting_hours=15, hours_per_day=0, work=[(50,25,25)])



with open("./conf_project.yaml","w") as f:
    for line in lines:
        
        if("Workloads_rs: [" in line):
            
            f.write("Workloads_rs: ["+str(work)+"]\n\n")
        else:
            f.write(line)


50000 (10000, 5) 100 500.0
